# Renta de Bicicletas

## Prediccion de la demanda en el alquiler de bicicletas

**librerias**

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from math import sqrt
import pickle

import matplotlib.pyplot as plt
%matplotlib inline


from sklearn.linear_model import LinearRegression as LinReg
from sklearn.preprocessing import PolynomialFeatures as PF
from sklearn.ensemble import GradientBoostingRegressor as GBR


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

## Datos

UCI's Bike Sharing Dataset Data Set
[link](https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset)

<B>Resumen:</B> Este dataset contiene las rentas de bicicletas por hora y dia entre los años 2011 y 2012 en el sistema Capital Bikeshare con la correspondiente informacion climatologica y estacional.

**Atributos:**


Ambos csv tienen los siguientes campos, excepto hr que no esta en day.csv

- **instant**: indice
- **dteday** : fecha
- **season** : estacion (1:primavera, 2:verano, 3:otoño, 4:invierno)
- **yr** : año (0: 2011, 1:2012)
- **mnth** : mes ( 1 to 12)
- **hr** : hora (0 to 23)
- **holiday** : dia festivo o no
- **weekday** : dia de la semana
- **workingday** : dia laborable o no
+ **weathersit** : 
    - 1: Despejado, Pocas nubes, Parcialmente nublado
    - 2: Niebla, Niebla+Nublado
    - 3: Nieve ligera, LLuvia ligera+Tormenta+Nube dispersa
    - 4: LLuvia fuerte+Hielo+Tormenta+Niebla,Nieve+Niebla
- **temp** : temperatura normalizada en grados centigrados. MinMax Scaler (t-t_min)/(t_max-t_min), t_min=-8, t_max=+39 (solo en escala horaria)
- **atemp**: sensacion termica normalizada en grados centigrados. MinMax Scaler (t-t_min)/(t_max-t_min), t_min=-16, t_max=+50 (solo en escala horaria)
- **hum**: humedad normalizada, dividido entre 100 (max)
- **windspeed**: velocidad del viento normalizada, dividido entre 67 (max)
- **casual**: cuenta de usuarios casuales
- **registered**: cuenta de usuarios registrados
- **cnt**: cuenta total del alquiler de bicicletas, incluyendo tanto los casuales como los registrados

